In [1]:
import numpy as np
import pandas as pd

In [2]:
from features import *

In [3]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
df = pd.read_csv('/hkfs/work/workspace/scratch/bh6321-energy_challenge/data/train.csv')

In [9]:
# df_val = pd.read_csv('/hkfs/work/workspace/scratch/bh6321-energy_challenge/data/valid.csv')
# df = pd.concat([df, df_val])

In [10]:
df = preprocess(df)
df = add_time_features(df, drop = True)
df_list = add_ts_features(df, return_as_list=True)
df_list = [df[df.year != 2015].copy() for df in df_list]

In [11]:
# add target by shifting load by 7*24 hours (separately for each city!)
for df in df_list:
    #df['target'] = df.groupby('city')['load'].shift(-7*24)
    df["target"] = df["load"].shift(-7*24)
    df.dropna(inplace = True)

In [18]:
sun_features = ["latitute", "longitude", "elevation", "is_day", "is_dusk", "is_dawn", "sun_position"]

In [19]:
df_list[0].columns

Index(['load', 'city', 'hour', 'day_name', 'weekday', 'day', 'month', 'year',
       'dayofyear', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'dayofyear_sin', 'dayofyear_cos', 'load_sq', 'load_t-1',
       'load_t-2', 'load_rmean', 'load_rstd', 'load_rmin', 'load_rmax',
       'target'],
      dtype='object')

In [22]:
FEATURES = list(df_list[0].columns)
FEATURES.remove("target")
FEATURES.remove("day_name")
FEATURES.remove("city")
#FEATURES.remove("time")
#for f in sun_features:
#    print(f)
#    FEATURES.remove(f)

## Random Forest

In [25]:
regressors = {}

for df_tmp in df_list:
    
    city = df_tmp.city.unique()[0]
    
    print("City: ", city)
    
    X = df_tmp[FEATURES]
    y = df_tmp["target"]
    
    regr = RandomForestRegressor(n_estimators=2000,
                                 max_depth=20, 
                                 criterion="squared_error",
                                 max_features="sqrt",
                                 random_state=7531,
                                 oob_score=False,
                                 n_jobs=-1)
    
    
    regr.fit(X,y)
    
    regressors[city] = regr
print("DONE")

City:  bs
City:  h
City:  hm
City:  ol
City:  del
City:  el
City:  os
City:  wob
City:  hi
City:  whv
City:  lg
City:  ce
City:  go
City:  sz
DONE


In [ ]:
import pickle

In [52]:
with open('model_dict_.pickle', 'wb') as handle:
    pickle.dump(regressors, handle, protocol=pickle.HIGHEST_PROTOCOL)